# Preparing environment

In [ ]:
!pip install -q datasets transformers

     |████████████████████████████████| 194kB 18.5MB/s 
     |████████████████████████████████| 2.1MB 47.8MB/s 
     |████████████████████████████████| 112kB 61.6MB/s 
     |████████████████████████████████| 245kB 40.8MB/s 
     |████████████████████████████████| 3.3MB 56.3MB/s 
     |████████████████████████████████| 870kB 51.9MB/s 


In [ ]:
from sklearn.metrics import accuracy_score
from datasets import load_dataset
import transformers

# Model loading

In [ ]:
MODEL = 'distilbert-base-uncased'

bert = transformers.AutoModelForSequenceClassification.from_pretrained(MODEL,num_labels=2)

bert_tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

# Dataset loading

### downloading

In [ ]:
dataset = load_dataset('amazon_reviews_multi', 'en')

train_dataset = dataset['train']
valid_dataset = dataset['validation']
test_dataset = dataset['test']

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/f3357bd271e187385a38574fe31b8fb10055303f67fa9fce55e84d08c4870efd. Subsequent calls will reuse this data.


### tokenize

In [ ]:
def tokenizer(data):
  return bert_tokenizer(
    data['review_body'],
    padding=True,
    truncation=True, 
    max_length=40, 
    pad_to_max_length=True,
  )

train_dataset = train_dataset.map(tokenizer, batched=True)
valid_dataset = valid_dataset.map(tokenizer, batched=True)
test_dataset = test_dataset.map(tokenizer, batched=True)

### set `stars` to 2-classes: `bad` or `ok`

In [ ]:
def limiter(data):
  data['label'] = 0 if data['stars'] <= 3 else 1 
  return data

train_dataset = train_dataset.map(limiter)
valid_dataset = valid_dataset.map(limiter)
test_dataset  = test_dataset.map(limiter)

train_dataset.set_format('torch')
valid_dataset.set_format('torch')
test_dataset.set_format('torch')

# Training

In [ ]:
batch_size = 512
num_train_epochs = 3

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
        }

args = transformers.TrainingArguments(
    output_dir = "tmp",
    evaluation_strategy = "epoch",
    learning_rate = 1e-5,
    per_device_train_batch_size = batch_size,
    warmup_steps = 10,
    num_train_epochs = num_train_epochs,
)

trainer = transformers.Trainer(
    bert,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,No log,0.321024,0.868200,20.610900,242.590000
2,0.352500,0.303810,0.873400,21.111000,236.843000
3,0.297400,0.300918,0.874200,20.605100,242.658000


TrainOutput(global_step=1173, training_loss=0.3198486386662554, metrics={'train_runtime': 3203.3622, 'train_samples_per_second': 0.366, 'total_flos': 9641521440000000.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2609684480, 'init_mem_gpu_alloc_delta': 268953088, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 150790144, 'train_mem_gpu_alloc_delta': 807732736, 'train_mem_cpu_peaked_delta': 266240, 'train_mem_gpu_peaked_delta': 7663377408})

# Evaluating

In [ ]:
trainer.evaluate(test_dataset)

{'epoch': 3.0,
 'eval_accuracy': 0.8678,
 'eval_loss': 0.3097822368144989,
 'eval_mem_cpu_alloc_delta': -4096,
 'eval_mem_cpu_peaked_delta': 4096,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 17060352,
 'eval_runtime': 20.114,
 'eval_samples_per_second': 248.583}